In [1]:
import os    
from copy import deepcopy
import codecs
import glob
import csv
import re
import numpy as np
import random
from sklearn.base import TransformerMixin
from sklearn.pipeline import make_pipeline, make_union
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.cross_validation import train_test_split


In [2]:
files = os.listdir('./data/')

In [3]:
deep_list = ['/*' * count for count in range(10)]

In [4]:
ext_list = [".gcc", ".c",".sbcl",".hack", ".php",".jruby",".yarv",".racket",".clojure",
            ".csharp",".java",".ocaml",".perl",".javascript",".python3",".scala",".ghc"]

In [5]:
this = [(glob.glob('./data/benchmarksgame-2014-08-31/{}{}'.format(x,ext)), ext) for x in deep_list for ext in ext_list]

In [6]:
final_list = []
for x,y in this:
    if x != []:
        final_list.append((x,y))

In [7]:
snippit_dict = {}
for itm, idx in final_list:
    if idx in snippit_dict.keys():
        snippit_dict[idx].append(itm)
    else:
        snippit_dict[idx] = itm


In [8]:
for key in snippit_dict:
    lan_list = []
    for string in snippit_dict[key]:
        try:
            with open(string, encoding='windows 1252') as f:
                lan_list.append(f.read())
        except:
            pass
    snippit_dict[key] = lan_list

In [9]:
snippit_dict['c'] = snippit_dict['.c'] + snippit_dict['.gcc']
snippit_dict['ruby'] = snippit_dict['.jruby'] + snippit_dict['.yarv']
snippit_dict['python'] = snippit_dict['.python3']
snippit_dict['php'] = snippit_dict['.hack'] + snippit_dict['.php']
snippit_dict['scala'] = snippit_dict['.scala']
snippit_dict['ocaml'] = snippit_dict['.ocaml']
snippit_dict['c#'] = snippit_dict['.csharp']
snippit_dict['lisp'] = snippit_dict['.sbcl']
snippit_dict['haskell'] = snippit_dict['.ghc']
snippit_dict['clojure'] = snippit_dict['.clojure']
snippit_dict['java'] = snippit_dict['.java']
snippit_dict['javascript'] = snippit_dict['.javascript']
snippit_dict['perl'] = snippit_dict['.perl']
snippit_dict['scheme'] = snippit_dict['.racket']

In [10]:
for string in ext_list:
    del snippit_dict[string]

In [11]:
X = []
y = []
for key in snippit_dict:
    y.extend([key for _ in range(len(snippit_dict[key]))])
    X.extend(snippit_dict[key])

In [12]:
# symbol_list = '''(
# )
# end
# ):
# else:
# :
# #
# /*
# */
# __init__
# __
# (ns
# @
# let
# elsif
# elif
# self.
# $
# my
# sub
# use
# {
# }
# (if 
# (defn
# private
# (defmacro
# *
# def 
# function
# ui
# print(
# var
# int
# do
# //
# final
# void
# mul
# done
# for
# while
# ;
# ;;
# import
# (def
# ){
# ) {
# public
# static
# native
# println
# Console
# Child
# <?
# (define
# (declaim
# Class'''.split('\n')

In [13]:
tup_list = [('(',')'),('[',']'),('{','}'),('<','>')]

In [14]:
symbol_list = '''Console
Child
(defn
(define
;;;
end
class
Class
object
case
()
==
__
import
var
puts
do
):
from
$
int
native
@
new
elif
public
private
set
static
void
final
public class
if
check
while
let
make
object
type
module
--
::
->
function
{
|
}
def'''.split('\n')

In [15]:
def percent_symbol(text,symbol):
    ret = len(re.findall(r''+re.escape(symbol),text))
    return ret/(len(text))
    

def percent_encased(text,symbol1, symbol2):
    count = 0
    ret = re.findall(r''+symbol1+'[\s\S]*'+symbol2,text)
    for item in ret:
        count += len(item)
    return count/(len(text))

In [16]:
class SymbolFeaturizer(TransformerMixin):
    def __init__(self, symbol, sym_list,encase, tup_list):
        self.symbol = symbol
        self.sym_list=sym_list
        self.encase = encase
        self.tup_list = tup_list
        
    def fit(self, X, y=None):
        '''All SciKit-learn compatible transformers and classifiers have the same
        interface. `fit` should always return the same object (self)'''
        return self
    
    def transform(self, X):
        '''Given a list of original data, return a list of feature vectors'''
        feature_vectors = []
        for x in X:
            feature_vector = [self.symbol(x,sym) for sym in self.sym_list] 
            feature_vectors.append(feature_vector)
#             feature_vector = [self.encase(x,tup[0],tup[1]) for tup in self.tup_list]
#             feature_vectors.extend(feature_vector)
        return np.array(feature_vectors)


In [17]:
# multi_feat = make_union(SymbolFeaturizer(percent_symbol,sym_list=symbol_list),EncasedFeaturizer(tup_list))

In [18]:
pipe = make_pipeline(SymbolFeaturizer(percent_symbol,symbol_list,percent_encased,tup_list),DecisionTreeClassifier())

In [19]:
# X_train, X_test, y_train, y_test = train_test_split(X,y)

In [20]:
# print(classification_report(pipe.predict(X_test), y_test))

In [21]:
# pipe.score(X_test,y_test)

In [22]:
tests = pd.read_csv('test.csv',header=None)

In [23]:
count = 0
del_list = []
for item in tests[1]:
    if item == 'tcl':
        del_list.append(count)
    count += 1

In [24]:
for row in del_list:
    tests = tests.drop(row)

In [25]:
test_X = []
test_y = []
for key in tests[0]:
    with open('test/{}'.format(key)) as f:
        test_X.append(f.read())
        

In [26]:
pipe.fit(X,y)

Pipeline(steps=[('symbolfeaturizer', <__main__.SymbolFeaturizer object at 0x11293cd30>), ('decisiontreeclassifier', DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            random_state=None, splitter='best'))])

In [27]:
pipe.score([test for test in test_X],tests[1])

0.73333333333333328

In [28]:
check = pipe.predict([test for test in test_X])

In [29]:
count = 0
for item in check:
    if count == 25:
        count += 2
    print(item,tests[1][count])
    count += 1

clojure clojure
clojure clojure
ruby clojure
clojure clojure
python python
python python
python python
scala python
javascript javascript
javascript javascript
scala javascript
javascript javascript
scheme ruby
ruby ruby
ruby ruby
php haskell
ruby haskell
haskell haskell
scheme scheme
scheme scheme
php scheme
java java
java java
haskell scala
scala scala
php php
php php
php php
ocaml ocaml
ocaml ocaml


In [30]:
print(classification_report(pipe.predict([test for test in test_X]), tests[1]))

             precision    recall  f1-score   support

    clojure       0.75      1.00      0.86         3
    haskell       0.33      0.50      0.40         2
       java       1.00      1.00      1.00         2
 javascript       0.75      1.00      0.86         3
      ocaml       1.00      1.00      1.00         2
        php       1.00      0.60      0.75         5
     python       0.75      1.00      0.86         3
       ruby       0.67      0.50      0.57         4
      scala       0.50      0.33      0.40         3
     scheme       0.67      0.67      0.67         3

avg / total       0.75      0.73      0.72        30

